In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
from az_table import query_entities_values

In [2]:
# # # # # # #
# Config    #
# # # # # # #

load_dotenv()
connection_string = os.getenv("AZURE_TABLE_CONNECTION_STRING")
azure_table_name = os.getenv("AZURE_TABLE_NAME")
icao24 = os.getenv("ICAO24")

In [3]:
def get_stored_flight_data(icao24):
    partition_key = str("'"+icao24+"'")
    filter = "PartitionKey eq {}".format(partition_key)
    select = u"PartitionKey, time, longitude, latitude, on_ground, geo_altitude"
    data = query_entities_values(connection_string=connection_string,table_name=azure_table_name, select=select, filter=filter)
    return data

In [4]:
def clean_and_sort(data):
    lst = []
    for item in data:
        if item['time'] != None:
            lst.append(item)
        else:
            pass
    sorted_data = sorted(lst, key=lambda d: d['time'])
    return sorted_data

In [5]:
def transform_data(data):
    #df = pd.DataFrame(data, columns=["PartitionKey", "time", "longitude", "latitude", "on_ground", "geo_altitude"])
    df = pd.DataFrame(data, columns=["longitude", "latitude"])
    df = df.drop_duplicates(subset=["longitude", "latitude"], keep='last')
    return df

In [6]:
def plot(df):
    BBox = (df.longitude.min(), df.longitude.max(), df.latitude.min(), df.latitude.max())
    # initialize an axis
    fig, ax = plt.subplots(figsize=(8,6))
    # plot map on axis
    countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    countries[countries["name"] == "Sweden"].plot(color="lightgrey", ax=ax)
    # plot points
    df.plot(x="longitude", y="latitude", kind="scatter", c="brightness", colormap="YlOrRd", title=f"Flight", ax=ax)
    # add grid
    ax.grid(b=True, alpha=0.5)
    plt.show()